In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
matches_raw_dataset =  pd.read_csv("../datasets/matches.csv")
deliveries_raw_dataset =  pd.read_csv("../datasets/deliveries.csv")

In [3]:
deliveries_raw_dataset.head()

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,extras_type,is_wicket,player_dismissed,dismissal_kind,fielder
0,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,1,SC Ganguly,P Kumar,BB McCullum,0,1,1,legbyes,0,NaN,NaN,NaN
1,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,2,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
2,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,3,BB McCullum,P Kumar,SC Ganguly,0,1,1,wides,0,NaN,NaN,NaN
3,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,4,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN
4,335982,1,Kolkata Knight Riders,Royal Challengers Bangalore,0,5,BB McCullum,P Kumar,SC Ganguly,0,0,0,NaN,0,NaN,NaN,NaN


# Creating Players Dataset

In [4]:
all_players = pd.concat([
    deliveries_raw_dataset['batter'],
    deliveries_raw_dataset['bowler'],
    deliveries_raw_dataset['non_striker']
])
unique_players = all_players.dropna().unique()

players_df = pd.DataFrame({
    'player_id': range(1, len(unique_players) + 1),
    'player_name': unique_players,
    'country': None,
    'played_international':None,
    'bowling_style':None,
    'batting_style':None,
    'player_role':None
})


players_df.head()

players_df.to_csv('../datasets/players_df.csv', index=False)

,player_id,player_name,country,played_international,bowling_style,batting_style,player_role
0,1,SC Ganguly,None,None,None,None,None
1,2,BB McCullum,None,None,None,None,None
2,3,RT Ponting,None,None,None,None,None
3,4,DJ Hussey,None,None,None,None,None
4,5,Mohammad Hafeez,None,None,None,None,None


# Scraping Player Data

In [5]:
import requests
from bs4 import BeautifulSoup

In [6]:
# Define the URL
url = "https://www.espncricinfo.com/records/trophy/indian-premier-league-117"

# Send a GET request
response = requests.get(url)
if response.status_code == 200:
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the "Player averages" section by its title
    player_averages_section = soup.find('span', string='Player averages')
    if player_averages_section:
        # Locate the parent container
        parent_div = player_averages_section.find_parent('div', class_="ds-w-full ds-bg-fill-content-prime ds-overflow-hidden ds-rounded-xl ds-border ds-border-line ds-mb-2")
        if parent_div:
            # Find all links within the section
            links = parent_div.find_all('a', href=True)
            
            # Prepare data for DataFrame
            team_links = []
            for link in links:
                team_name = link.find('span').text.strip() if link.find('span') else None
                full_link = "https://www.espncricinfo.com" + link['href']
                if team_name:  # Avoid empty team names
                    team_links.append({'Team': team_name, 'Link': full_link})
            
            # Create a DataFrame
            team_links_df = pd.DataFrame(team_links)
        else:
            print("Could not locate the parent container for 'Player averages'.")
    else:
        print("'Player averages' section not found.")
else:
    print(f"Failed to fetch the webpage. Status code: {response.status_code}")



In [7]:
team_links_df.to_csv('../datasets/team_links_df.csv', index=False)

In [8]:
team_links_df.head()

,Team,Link
0,Chennai Super Kings,https://www.espncricinfo.com/records/trophy/av...
1,Deccan Chargers,https://www.espncricinfo.com/records/trophy/av...
2,Delhi Daredevils,https://www.espncricinfo.com/records/trophy/av...
3,Gujarat Lions,https://www.espncricinfo.com/records/trophy/av...
4,Gujarat Titans,https://www.espncricinfo.com/records/trophy/av...


## Getting Player IPL Team and played time

In [9]:
def get_data_from_table(response):
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'class': 'ds-w-full ds-table ds-table-xs ds-table-auto ds-w-full ds-overflow-scroll ds-scrollbar-hide'})
    
    # Check if the table exists
    if not table:
        return pd.DataFrame(columns=['Player', 'Span', 'Mat'])
    
    # Extract table rows
    rows = table.find('tbody').find_all('tr')
    
    # Extract required data
    data = []
    for row in rows:
        cols = row.find_all('td')
        player = cols[0].text.strip()
        span = cols[1].text.strip()
        mat = cols[2].text.strip()
        data.append([player, span, mat])
    
    return pd.DataFrame(data, columns=['Player', 'Span', 'Mat'])

# Master DataFrame to collect all data
master_df = pd.DataFrame(columns=['Player', 'Span', 'Mat', 'Team'])

# Loop through each team's link
for index, row in team_links_df.iterrows():
    team = row['Team']
    url = row['Link']
    
    # Fetch the webpage
    response = requests.get(url)
    
    # Extract data from the table
    team_df = get_data_from_table(response)
    
    # Add the team name to the DataFrame
    team_df['Team'] = team
    
    # Append to the master DataFrame
    master_df = pd.concat([master_df, team_df], ignore_index=True)

# Display the consolidated DataFrame
print(master_df)

                 Player       Span Mat                 Team
0            P Amarnath  2008-2008   6  Chennai Super Kings
1           Akash Singh  2023-2023   6  Chennai Super Kings
2                MM Ali  2021-2024  48  Chennai Super Kings
3            S Anirudha  2008-2013  19  Chennai Super Kings
4       KB Arun Karthik  2010-2010   1  Chennai Super Kings
...                 ...        ...  ..                  ...
2767          DA Warner  2014-2021  95  Sunrisers Hyderabad
2768  Washington Sundar  2022-2024  18  Sunrisers Hyderabad
2769           CL White  2013-2013  13  Sunrisers Hyderabad
2770      KS Williamson  2015-2022  76  Sunrisers Hyderabad
2771       Yuvraj Singh  2016-2017  22  Sunrisers Hyderabad

[2772 rows x 4 columns]


In [10]:
master_df_cleaned = master_df.drop_duplicates(subset=['Player', 'Span', 'Mat'], keep='first')
print(master_df_cleaned)
master_df_cleaned.to_csv('../datasets/players_ipl_summary_dataset.csv', index=False)

                 Player       Span Mat                 Team
0            P Amarnath  2008-2008   6  Chennai Super Kings
1           Akash Singh  2023-2023   6  Chennai Super Kings
2                MM Ali  2021-2024  48  Chennai Super Kings
3            S Anirudha  2008-2013  19  Chennai Super Kings
4       KB Arun Karthik  2010-2010   1  Chennai Super Kings
...                 ...        ...  ..                  ...
1381          DA Warner  2014-2021  95  Sunrisers Hyderabad
1382  Washington Sundar  2022-2024  18  Sunrisers Hyderabad
1383           CL White  2013-2013  13  Sunrisers Hyderabad
1384      KS Williamson  2015-2022  76  Sunrisers Hyderabad
1385       Yuvraj Singh  2016-2017  22  Sunrisers Hyderabad

[1386 rows x 4 columns]


## Getting Player Profile Links

In [11]:
def get_data_from_table(response):
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'class': 'ds-w-full ds-table ds-table-xs ds-table-auto ds-w-full ds-overflow-scroll ds-scrollbar-hide'})
    
    # Check if the table exists
    if not table:
        return pd.DataFrame(columns=['Player', 'Link'])
    
    # Extract table rows
    rows = table.find('tbody').find_all('tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        player_col = cols[0]
        # Extract player name
        player = player_col.text.strip()
        
        # Extract player link
        link = player_col.find('a')['href'] if player_col.find('a') else None
        
        data.append([player, link])
        
    return pd.DataFrame(data, columns=['Player', 'Link'])
    
# Master DataFrame to collect all data
master_df = pd.DataFrame(columns=['Player', 'Link'])    
    
team_links_df = pd.read_csv('../datasets/team_links_df.csv')

for index, row in team_links_df.iterrows():
    team = row['Team']
    url = row['Link']

    # Fetch the webpage
    response = requests.get(url)
    
    # Extract data from the table
    team_df = get_data_from_table(response)
    
    # Append to the master DataFrame
    master_df = pd.concat([master_df, team_df], ignore_index=True)


# Display the consolidated DataFrame
print(master_df)

                 Player                                  Link
0            P Amarnath    /cricketers/palani-amarnath-290629
1           Akash Singh       /cricketers/akash-singh-1175458
2                MM Ali            /cricketers/moeen-ali-8917
3            S Anirudha  /cricketers/srikkanth-anirudha-26798
4       KB Arun Karthik       /cricketers/arun-karthik-319745
...                 ...                                   ...
2767          DA Warner       /cricketers/david-warner-219889
2768  Washington Sundar  /cricketers/washington-sundar-719715
2769           CL White        /cricketers/cameron-white-8291
2770      KS Williamson    /cricketers/kane-williamson-277906
2771       Yuvraj Singh        /cricketers/yuvraj-singh-36084

[2772 rows x 2 columns]


In [12]:
master_df_cleaned = master_df.drop_duplicates(subset=['Player', 'Link'], keep='first')
print(master_df_cleaned)
master_df_cleaned.to_csv('../datasets/players_profile_links_dataset.csv', index=False)

                    Player                                        Link
0               P Amarnath          /cricketers/palani-amarnath-290629
1              Akash Singh             /cricketers/akash-singh-1175458
2                   MM Ali                  /cricketers/moeen-ali-8917
3               S Anirudha        /cricketers/srikkanth-anirudha-26798
4          KB Arun Karthik             /cricketers/arun-karthik-319745
...                    ...                                         ...
1372  X Thalaivan Sargunam       /cricketers/thalaivan-sargunam-319744
1374           Umran Malik             /cricketers/umran-malik-1246528
1378           Virat Singh              /cricketers/virat-singh-720465
1379        Vivrant Sharma          /cricketers/vivrant-sharma-1252370
1380          V Viyaskanth  /cricketers/vijayakanth-viyaskanth-1153149

[735 rows x 2 columns]


In [13]:
master_df_cleaned.to_csv('../datasets/players_profile_links_dataset.csv', index=False)

## Scraping Player Profile information

In [14]:
def get_player_information(response):
    soup = BeautifulSoup(response.content, 'html.parser')

    target_div = soup.find('div', class_='ds-pt-8 ds-px-6 ds-pb-2 ds-text-raw-white')

    # Initialize default values in case the div is not found
    player_name = None
    country = None
    player_role = None
    intl_career = None
    batting_style = None
    bowling_style =  None
    
    if target_div:
        # Extract the player's name
        name_element = target_div.find('h1', class_='ds-text-title-l ds-font-bold')
        player_name = name_element.text.strip() if name_element else None
        
        # Extract the country and role
        span_elements = target_div.find_all('span', class_='ds-text-comfortable-s')
        if len(span_elements) >= 2:
            country = span_elements[0].text.strip()
            player_role = span_elements[1].text.strip()
            
    intl_career_div = soup.find('div', class_='ds-bg-ui-fill-contrast ds-px-4 ds-py-2 ds-text-raw-grey06')
    if intl_career_div:
        # Extract the career span
        career_span_element = intl_career_div.find('span', class_='ds-text-comfortable-s')
        if career_span_element:
            intl_career = career_span_element.text.replace("INTL CAREER: ", "").strip()
    
    overview_div = soup.find('div', class_='ds-p-4')        
    # Extract Batting Style
    batting_style = overview_div.find('p', string='Batting Style')
    if batting_style:
        batting_style = batting_style.find_next('p').text

    bowling_style_element  = overview_div.find('p', string='Bowling Style')
    if bowling_style_element :  
        bowling_style = bowling_style_element.find_next('p').text
    
    
    # Return the extracted data
    player = {
        'player_name': player_name,
        'country': country,
        'player_role': player_role,
        'intl_career': intl_career,
        'batting_style':batting_style,
        'bowling_style':bowling_style
        
    }
    return player

player_profile_df =  pd.read_csv('../datasets/players_profile_links_dataset.csv')
players_df =  pd.read_csv('../datasets/players_dataset.csv')

players_df = players_df.astype({
    "country": "object",
    "batting_style": "object",
    "bowling_style": "object",
    "player_role": "object",
    "played_international": "object"
})

for index, row in player_profile_df.iterrows():
    player_short_name = row["Player"]
    link = row["Link"]
    full_link = "https://www.espncricinfo.com" + link
    response = requests.get(full_link)
    player = get_player_information(response)
    player["short_name"] = player_short_name
    if player_short_name in players_df["player_name"].values:
        # Locate the matching row in players_df
        player_index = players_df[players_df["player_name"] == player_short_name].index[0]
        
        # Update the relevant columns in players_df
        players_df.at[player_index, "country"] = player["country"]
        players_df.at[player_index, "played_international"] = player["intl_career"]
        players_df.at[player_index, "batting_style"] = player["batting_style"]
        players_df.at[player_index, "bowling_style"] = player["bowling_style"]
        players_df.at[player_index, "player_role"] = player["player_role"]
        
players_df.to_csv('../datasets/players_info_summary_new.csv', index=False)
    
    